## Importation Biblio

In [2]:
import pandas as pd
from rdflib import Graph
from pathlib import Path

## Nettoyage + EDA 

## Chargement des dateset (philadelphia)

In [3]:
# Ridership par mode
df_mode = pd.read_csv(
    r"C:\Fil Blanc\philadelphia\Average_Daily_Ridership_By_Mode - City of Philadelphia.csv"
)



df_mode = df_mode.rename(columns={
    "Mode": "mode",
    "Average_Daily_Ridership": "ridership",
    "Calendar_Year": "year",
    "Calendar_Month": "month"
})


df_mode = df_mode[
    [ "year", "month", "mode", "ridership"]

]

df_mode.info()
df_mode.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492 entries, 0 to 491
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   year       492 non-null    int64 
 1   month      492 non-null    int64 
 2   mode       492 non-null    object
 3   ridership  492 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 15.5+ KB


,year,month,mode,ridership
0,2019,1,Bus,459160
1,2019,1,CCT,4294
2,2019,1,Heavy Rail,296709
3,2019,1,Regional Rail,122856
4,2019,1,Trackless Trolley,16853


In [4]:
# Vérifier les valeurs manquantes dans ridership par mode
df_mode.isna().sum()

year         0
month        0
mode         0
ridership    0
dtype: int64

In [5]:

# Ridership quotidien par route
df_route = pd.read_csv(
    r"C:\Fil Blanc\philadelphia\Average_Daily_Ridership_By_Route - City of Philadelphia.csv"
)


df_route = df_route.rename(columns={
    "Route": "route_id",
    "Average_Daily_Ridership": "ridership",
    "Calendar_Year": "year",
    "Calendar_Month": "month"
})


df_route = df_route[
    [ "year", "month", "route_id", "ridership"]
]

df_route.info()
df_route.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10994 entries, 0 to 10993
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   year       10994 non-null  int64 
 1   month      10994 non-null  int64 
 2   route_id   10994 non-null  object
 3   ridership  10994 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 343.7+ KB


,year,month,route_id,ridership
0,2019,8,59,3811
1,2019,8,60,8550
2,2019,8,61,3563
3,2019,8,62,226
4,2019,8,64,4335


In [6]:
# Vérifier les valeurs manquantes dans ridership par route
df_route.isna().sum()

year         0
month        0
route_id     0
ridership    0
dtype: int64

In [7]:
# Vérifier les doublons dans ridership par mode
df_mode.duplicated().sum()


np.int64(0)

In [8]:
# Vérifier les doublons dans ridership par route
df_route.duplicated().sum()

np.int64(0)

In [9]:
# Sauvegarder les données nettoyées
df_mode.to_csv("data_cleaned/ridership_philadelphia_mode.csv", index=False)
df_route.to_csv("data_cleaned/ridership_philadelphia_route.csv", index=False)

## Chargement des dateset (Chicago)

In [10]:
df_chicago_mode = pd.read_excel('C:\Fil Blanc\cta-ridership-daily-boarding-totals-20260203-69820a3f9df63091665572.xlsx')
df_chicago_mode.info()
df_chicago_mode.value_counts('service_date')

<>:1: SyntaxWarning: "\F" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\F"? A raw string is also an option.
<>:1: SyntaxWarning: "\F" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\F"? A raw string is also an option.
C:\Users\imane\AppData\Local\Temp\ipykernel_3500\1613795136.py:1: SyntaxWarning: "\F" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\F"? A raw string is also an option.
  df_chicago_mode = pd.read_excel('C:\Fil Blanc\cta-ridership-daily-boarding-totals-20260203-69820a3f9df63091665572.xlsx')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9100 entries, 0 to 9099
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   service_date    9100 non-null   datetime64[ns]
 1   day_type        9100 non-null   object        
 2   bus             9100 non-null   int64         
 3   rail_boardings  9100 non-null   int64         
 4   total_rides     9100 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 355.6+ KB


service_date
2001-01-01    1
2001-01-02    1
2001-01-03    1
2001-01-04    1
2001-01-05    1
             ..
2025-11-26    1
2025-11-27    1
2025-11-28    1
2025-11-29    1
2025-11-30    1
Name: count, Length: 9100, dtype: int64

In [11]:
df_chicago_mode["service_date"] = pd.to_datetime(df_chicago_mode["service_date"])
df_chicago_mode["year"] = df_chicago_mode["service_date"].dt.year
df_chicago_mode["month"] = df_chicago_mode["service_date"].dt.month

# Filtrer 2019 → 2025
df_chicago_mode = df_chicago_mode[
    (df_chicago_mode['service_date'] >= '2019-01-01') &
    (df_chicago_mode['service_date'] <= '2025-12-31')
]

# Agréger par mois
df_chicago_mode_monthly = (
    df_chicago_mode
    .groupby(["year", "month"], as_index=False)
    .agg({
        "bus": "sum",
        "rail_boardings": "sum"
    })
)

In [12]:
df_chicago_mode.value_counts('service_date')
# df_chicago_mode.head()
df_chicago_mode_monthly.head()
df_chicago_mode_monthly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   year            83 non-null     int32
 1   month           83 non-null     int32
 2   bus             83 non-null     int64
 3   rail_boardings  83 non-null     int64
dtypes: int32(2), int64(2)
memory usage: 2.1 KB


In [ ]:
# Vérifier les doublons dans ridership par mode à Chicago
df_chicago_mode_monthly.duplicated().sum()

np.int64(0)

In [14]:
df_chicago_mode_monthly.to_csv("data_cleaned/ridership_chicago_mode.csv", index=False)


## Convertir RDF to CSV

In [15]:
import pandas as pd
import glob
from lxml import etree

# Chemin vers tes fichiers RDF
RDF_PATH = "C:\\Fil Blanc\\chicagoRDF\\*.rdf"

# Namespaces RDF
NS = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "ds": "https://data.cityofchicago.org/resource/jyb9-n7fm/"
}

def parse_rdf(file_path):
    tree = etree.parse(file_path)
    root = tree.getroot()

    records = []

    for desc in root.findall("rdf:Description", namespaces=NS):
        record = {
            "route": desc.findtext("ds:route", namespaces=NS),
            "date": desc.findtext("ds:date", namespaces=NS),
            "daytype": desc.findtext("ds:daytype", namespaces=NS),
            "rides": desc.findtext("ds:rides", namespaces=NS),
        }
        records.append(record)

    return pd.DataFrame(records)

# Lire tous les fichiers RDF
dfs = []
for file in glob.glob(RDF_PATH):
    print(f"Lecture : {file}")
    dfs.append(parse_rdf(file))

# Fusion
df_chicago_route = pd.concat(dfs, ignore_index=True)

# Nettoyage
df_chicago_route["date"] = pd.to_datetime(df_chicago_route["date"])
df_chicago_route["rides"] = df_chicago_route["rides"].astype(int)
df_chicago_route["route"] = df_chicago_route["route"].astype(str)

Lecture : C:\Fil Blanc\chicagoRDF\rdf_CTA__Ridership__Daily_by_Route_routes_1.rdf
Lecture : C:\Fil Blanc\chicagoRDF\rdf_CTA__Ridership__Daily_by_Route_routes_10.rdf
Lecture : C:\Fil Blanc\chicagoRDF\rdf_CTA__Ridership__Daily_by_Route_routes_11.rdf
Lecture : C:\Fil Blanc\chicagoRDF\rdf_CTA__Ridership__Daily_by_Route_routes_12.rdf
Lecture : C:\Fil Blanc\chicagoRDF\rdf_CTA__Ridership__Daily_by_Route_routes_13.rdf
Lecture : C:\Fil Blanc\chicagoRDF\rdf_CTA__Ridership__Daily_by_Route_routes_14.rdf
Lecture : C:\Fil Blanc\chicagoRDF\rdf_CTA__Ridership__Daily_by_Route_routes_15.rdf
Lecture : C:\Fil Blanc\chicagoRDF\rdf_CTA__Ridership__Daily_by_Route_routes_16.rdf
Lecture : C:\Fil Blanc\chicagoRDF\rdf_CTA__Ridership__Daily_by_Route_routes_17.rdf
Lecture : C:\Fil Blanc\chicagoRDF\rdf_CTA__Ridership__Daily_by_Route_routes_18.rdf
Lecture : C:\Fil Blanc\chicagoRDF\rdf_CTA__Ridership__Daily_by_Route_routes_19.rdf
Lecture : C:\Fil Blanc\chicagoRDF\rdf_CTA__Ridership__Daily_by_Route_routes_2.rdf
Lectur

In [16]:
df_chicago_route.info()
df_chicago_route.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1092474 entries, 0 to 1092473
Data columns (total 4 columns):
 #   Column   Non-Null Count    Dtype         
---  ------   --------------    -----         
 0   route    1092474 non-null  object        
 1   date     1092474 non-null  datetime64[ns]
 2   daytype  1092474 non-null  object        
 3   rides    1092474 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 33.3+ MB


,route,date,daytype,rides
0,1,2004-06-16,W,3174
1,100,2021-06-29,W,176
2,100,2018-05-21,W,497
3,106,2009-02-25,W,3146
4,100,2021-12-30,W,204


In [17]:

# Convertir la date
df_chicago_route["date"] = pd.to_datetime(df_chicago_route["date"])

# Créer année & mois
df_chicago_route["year"] = df_chicago_route["date"].dt.year
df_chicago_route["month"] = df_chicago_route["date"].dt.month

# Filtrer 2019 → 2025
df_chicago_route = df_chicago_route[
    (df_chicago_route["date"].dt.year >= 2019) &
    (df_chicago_route["date"].dt.year <= 2025)
]

# Agrégation mensuelle
df_chicago_route_monthly = (
    df_chicago_route
    .groupby(["year", "month", "route"], as_index=False)
    .agg({"rides": "sum"})
    .rename(columns={"rides": "ridership"})
)

In [18]:
df_chicago_route_monthly.duplicated().sum()

np.int64(0)

In [19]:
df_chicago_route_monthly.info()
# Export CSV
output_path = "data_cleaned/ridership_chicago_route.csv"
df_chicago_route_monthly.to_csv(output_path, index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10465 entries, 0 to 10464
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   year       10465 non-null  int32 
 1   month      10465 non-null  int32 
 2   route      10465 non-null  object
 3   ridership  10465 non-null  int64 
dtypes: int32(2), int64(1), object(1)
memory usage: 245.4+ KB


## Modélisation 

In [20]:

dim_year = pd.DataFrame({
    "year": list(range(2019, 2026))
})
dim_year.head(20)
# Export CSV
dim_year.to_csv("data_cleaned/dim_year.csv", index=False)

In [21]:
dim_month = pd.DataFrame({
    "month": list(range(1, 13)),
    "month_name": [
        "Jan", "Feb", "Mar", "Apr", "May", "Jun",
        "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"
    ],
    "month_order": list(range(1, 13))
})

dim_month.head(20)
# Export CSV
dim_month.to_csv("data_cleaned/dim_month.csv", index=False)